In [1]:
from __future__ import print_function
import os

import numpy as np
import zipfile
import tarfile
from six.moves.urllib.request import urlretrieve
import shutil 
import random

import string
import tensorflow as tf

# Dirs - must be absolute paths!
LOG_DIR = '/tmp/tf/ptb_char_lstm/'
# Local dir where PTB files will be stored.
PTB_DIR = '/home/tkornuta/data/ptb/'

# Filenames.
TRAIN = "ptb.train.txt"
VALID = "ptb.valid.txt"
TEST = "ptb.test.txt"

# A single recurrent layer of 2000 units
#NUM_UNITS = 100
# Size of the hidden state 64
HIDDEN_SIZE = 64

# A batch size of 100
BATCH_SIZE = 100

# Sequences of length 200 bytes
SEQ_LENGTH = 5


### Check/maybe download PTB.

In [2]:
def maybe_download_ptb(path, 
                       filename='simple-examples.tgz', 
                       url='http://www.fit.vutbr.cz/~imikolov/rnnlm/', 
                       expected_bytes =34869662):
  # Eventually create the PTB dir.
  if not tf.gfile.Exists(path):
    tf.gfile.MakeDirs(path)
  """Download a file if not present, and make sure it's the right size."""
  _filename = path+filename
  if not os.path.exists(_filename):
    print('Downloading %s...' % filename)
    _filename, _ = urlretrieve(url+filename, _filename)
  statinfo = os.stat(_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', (_filename), '(', statinfo.st_size, ')')
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + _filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download_ptb(PTB_DIR)

Found and verified /home/tkornuta/data/ptb/simple-examples.tgz ( 34869662 )


### Extract dataset-related files from the PTB archive.

In [3]:
def extract_ptb(path, filename='simple-examples.tgz', files=["ptb.train.txt", "ptb.valid.txt", "ptb.test.txt", 
                                       "ptb.char.train.txt", "ptb.char.valid.txt", "ptb.char.test.txt"]):
    """Extracts files from PTB archive."""
    # Extract
    tar = tarfile.open(path+filename)
    tar.extractall(path)
    tar.close()
    # Copy files
    for file in files:
        shutil.copyfile(PTB_DIR+"simple-examples/data/"+file, PTB_DIR+file)
    # Delete directory
    shutil.rmtree(PTB_DIR+"simple-examples/")        

extract_ptb(PTB_DIR)

### Load train, valid and test texts.

In [4]:
def read_data(filename, path):
    with open(path+filename, 'r') as myfile:
        data=myfile.read()# .replace('\n', '')
        return data

train_text = read_data(TRAIN, PTB_DIR)
train_size=len(train_text)
print(train_size, train_text[:100])

valid_text = read_data(VALID, PTB_DIR)
valid_size=len(valid_text)
print(valid_size, valid_text[:64])

test_text = read_data(TEST, PTB_DIR)
test_size=len(test_text)
print(test_size, test_text[:64])

5101618  aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memote
399782  consumers may want to move their telephones a little closer to 
449945  no it was n't black monday 
 but while the new york stock excha


### Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = 59 # [A-Z] + [a-z] + ' ' +few 'in between; + punctuation
first_letter = ord(string.ascii_uppercase[0]) # ascii_uppercase before lowercase! 
print("vocabulary size = ", vocabulary_size)
print(first_letter)

def char2id(char):
  """ Converts char to id (int) with one-hot encoding handling of unexpected characters"""
  if char in string.ascii_letters:# or char in string.punctuation or char in string.digits:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    # print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  """ Converts single id (int) to character"""
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

#print(len(string.punctuation))
#for i in string.ascii_letters:
#    print (i, char2id(i))


print(char2id('a'), char2id('A'), char2id('z'), char2id('Z'), char2id(' '), char2id('ï'))
print(id2char(char2id('a')), id2char(char2id('A')))
#print(id2char(65), id2char(33), id2char(90), id2char(58), id2char(0))
#bankno
sample = np.zeros(shape=(1, vocabulary_size), dtype=np.float)
sample[0, char2id(' ')] = 1.0
print(sample)

vocabulary size =  59
65
33 1 58 26 0 0
a A
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.]]


### Helper class for batch generation

In [6]:
class BatchGenerator(object):
  def __init__(self, text, batch_size, seq_length, vocab_size):
    """
    Initializes the batch generator object. Stores the variables and first "letter batch".
    text is text to be processed
    batch_size is size of batch (number of samples)
    seq_length represents the length of sequence
    vocab_size is number of words in vocabulary (assumes one-hot encoding)
    """
    # Store input parameters.
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._seq_length = seq_length
    self._vocab_size = vocab_size
    # Divide text into segments depending on number of batches, each segment determines a cursor position for a batch.
    segment = self._text_size // batch_size
    # Set initial cursor position.
    self._cursor = [ offset * segment for offset in range(batch_size)]
    # Store first "letter batch".
    self._last_letter_batch = self._next_letter_batch()
  
  def _next_letter_batch(self):
    """
    Returns a batch containing of encoded single letters depending on the current batch 
    cursor positions in the data.
    Returned "letter batch" is of size batch_size x vocab_size
    """
    letter_batch = np.zeros(shape=(self._batch_size, self._vocab_size), dtype=np.float)
    # Iterate through "samples"
    for b in range(self._batch_size):
      # Set 1 in position pointed out by one-hot char encoding.
      letter_batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return letter_batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    # First add last letter from previous batch (the "additional one").
    batches = [self._last_letter_batch]
    for step in range(self._seq_length):
      batches.append(self._next_letter_batch())
    # Store last "letter batch" for next batch.
    self._last_letter_batch = batches[-1]
    return batches


In [7]:
# Trick - override first 10 chars
#list1 = list(train_text)
#for i in range(2):
#    list1[i] = 'z'
#train_text = ''.join(list1)
print("Train set =", train_text[0:100])

# Create objects for training, validation and testing batch generation.
train_batches = BatchGenerator(train_text, BATCH_SIZE, SEQ_LENGTH, vocabulary_size)
# The latter two - threat whole text as a batch.
valid_batches = BatchGenerator(valid_text, valid_size, 1, vocabulary_size)
test_batches = BatchGenerator(test_text, test_size, 1, vocabulary_size)

# Get first training batch.
batch = train_batches.next()
#print("Batch = ", batch)
print(batches2string(batch))
#print("batch len = num of enrollings",len(batch))
#for i in range(num_unrollings):
#    print("i = ", i, "letter=", batches2string(batch)[0][i][0], "bits = ", batch[i][0])

# Generate input-target pairs for validation set.
valid_batch = valid_batches.next()[0]
valid_batch_input = valid_batch[:valid_size-1]
valid_batch_target = valid_batch[1:valid_size]

# Generate input-target pairs for validation set.
test_batch = test_batches.next()[0]
test_batch_input = test_batch[:test_size-1]
test_batch_target = test_batch[1:test_size]



Train set =  aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memote
[' aer b', 'ere de', 'nts co', 'has ju', 'n N ac', 'upon r', ' tradi', 'ed the', 'rge of', 'nk  th', 's goin', 'levels', 'ays  u', 'ng los', 'rain  ', 'l bidd', 'lieves', 'elters', 'nk  my', 'ransac', ' of fi', 'ay its', 'commis', 'assed ', 'ld los', 'panies', ' mr  r', 'd mr  ', 've use', ' headq', ' sir j', 'ral ga', 'd the ', 'point ', ' which', 'learly', 'ested ', ' unk  ', 'e was ', 'o  unk', 'act th', 'idend ', 'e chil', 'mortga', 'icago ', 'd curr', 'rns ab', 'ysts a', 'r incl', 'er mes', 'millio', 'ars wo', 'nk  be', 'unkin ', 'g the ', 'nounce', 'p to t', 'has a ', 'ey say', ' outsi', '   jus', 'to rea', 'e fund', 'onth f', 'th an ', 'tegy b', 'd thei', 's guil', 'sumer ', 'd whic', 'ns of ', 'ritish', 'aurant', 'k    t', 'ensati', 'rd and', 'such t', 'before', 'east e', 'ailure', 'olving', 'attemp', 'ts   t', 'third ', 'e diff', 'tself ', 'l for ', 'er  s ', 'ery t

### Helper function defining the LSTM cell

In [8]:
  # Definition of the cell computation.
  def lstm_cell(i, o, state, name):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    with tf.name_scope(name):
        # Calculate gates activations.
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib, name="Input_gate")
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb, name="Forget_gate")
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob, name="Output_gate")

        update = tf.add(tf.matmul(i, cx), tf.matmul(o, cm) + cb, name="Update")
        state = tf.add(forget_gate * state, input_gate * tf.tanh(update), name="State_update")
        output = output_gate * tf.tanh(state)
        return output, state


###  Definition of tensor graph

In [9]:
# Reset graph - just in case.
tf.reset_default_graph()

# 0. Shared variables ops.
with tf.name_scope("Shared_Variables"):
  # Define parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, HIDDEN_SIZE], -0.1, 0.1), name="ix")
  im = tf.Variable(tf.truncated_normal([HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="im")
  ib = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="ib")
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, HIDDEN_SIZE], -0.1, 0.1), name="fx")
  fm = tf.Variable(tf.truncated_normal([HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="fm")
  fb = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="fb")
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, HIDDEN_SIZE], -0.1, 0.1), name="cx")
  cm = tf.Variable(tf.truncated_normal([HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="cm")
  cb = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="cb")
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, HIDDEN_SIZE], -0.1, 0.1), name="ox")
  om = tf.Variable(tf.truncated_normal([HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="om")
  ob = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="ob")
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([BATCH_SIZE, HIDDEN_SIZE]), trainable=False, name="saved_output")
  saved_state = tf.Variable(tf.zeros([BATCH_SIZE, HIDDEN_SIZE]), trainable=False, name="saved_state")
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([HIDDEN_SIZE, vocabulary_size], -0.1, 0.1), name="w")
  b = tf.Variable(tf.zeros([vocabulary_size]), name="b")
  
with tf.name_scope("Training"): 
    # 0. Placeholders for inputs.
    with tf.name_scope("Input_data"):
      # Define input data buffers.
      train_data = list()
      for _ in range(SEQ_LENGTH + 1):
        # Collect placeholders for inputs/labels.
        train_data.append(
          tf.placeholder(tf.float32, shape=[BATCH_SIZE,vocabulary_size], name="Input_data"))
      # Collection of training inputs.
      train_inputs = train_data[:SEQ_LENGTH]
      # Labels are pointing to the same placeholders!
      # Labels are inputs shifted by one time step.
      train_labels = train_data[1:]  
      print (len(train_inputs))
      print (train_inputs[0].shape)
      # Concatenate targets into 2D tensor.
      targets = tf.concat(train_labels, 0)


     # 2. Training LSTM ops.
    with tf.name_scope("LSTM"):
      # Unrolled LSTM loop.
      # Build outpus of size SEQ_LENGTH.
      outputs = list()
      output = saved_output
      state = saved_state
      for i in train_inputs:
        output, state = lstm_cell(i, output, state, "cell")
        outputs.append(output)
      print (len(outputs))
      print (outputs[0].shape)
      print (tf.concat(outputs, 0).shape)

      # State saving across unrollings.
      with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        # Fully connected layer on top => classification.
        # In fact we will create lots of FC layers (one for each output layer), with shared weights.
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)

    # 2. Loss ops.
    with tf.name_scope("Loss"):
        # Loss function(s) - one for every output generated by every lstm cell.
        loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
            labels=targets, logits=logits))
        # Add loss summary.
        loss_summary = tf.summary.scalar("loss", loss)

    # 3. Training ops.  
    with tf.name_scope("Optimization"):
      # Optimizer-related variables.
      global_step = tf.Variable(0)
      learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate)
      gradients, v = zip(*optimizer.compute_gradients(loss))
      gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
      optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    # 4. Predictions ops.  
    with tf.name_scope("Evaluation") as scope:
      # Predictions.
      train_prediction = tf.nn.softmax(logits)


5
(100, 59)
5
(100, 64)
(500, 64)


### Subgraph responsible for generation of sample texts, char by char.

In [10]:
with tf.name_scope("Sample_generation") as scope:
  # Create graphs for sampling and validation evaluation: batch 1, "no unrolling".
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size], name="Input_data")
  saved_sample_output = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="Output_data")
  saved_sample_state = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="Hidden_state")

  # Node responsible for resetting the state and output.
  reset_sample_state = tf.group(
      saved_sample_output.assign(tf.zeros([1, HIDDEN_SIZE])),
      saved_sample_state.assign(tf.zeros([1, HIDDEN_SIZE])))
  # Single LSTM cell.
  sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state, "cell")
  # Output depends on the hidden state.
  with tf.control_dependencies([saved_sample_output.assign(sample_output), saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b, name="logits"), name="outputs")

# Merge all summaries.
merged_summaries = tf.summary.merge_all()

# 4. Init global variable.
init = tf.global_variables_initializer() 

### Subgraph responsible for validation (1-char).

In [11]:
with tf.name_scope("Validation-1char"):
    with tf.name_scope("Input_data"):
      # Define buffers.
      valid_inputs = tf.placeholder(tf.float32, shape=[valid_size-1,vocabulary_size], name="Data")
      valid_targets = tf.placeholder(tf.float32, shape=[valid_size-1,vocabulary_size], name="Target")

    with tf.name_scope("LSTM"):
      prev_valid_outputs = tf.Variable(tf.zeros([valid_size-1, HIDDEN_SIZE]), name="Prev_Outputs")
      prev_valid_state = tf.Variable(tf.zeros([valid_size-1, HIDDEN_SIZE]), name="Prev_hidden_state")

      # Node responsible for resetting the state and output.
      reset_valid_state = tf.group(
          prev_valid_outputs.assign(tf.zeros([valid_size-1, HIDDEN_SIZE])),
          prev_valid_state.assign(tf.zeros([valid_size-1, HIDDEN_SIZE])))

      # Single LSTM cell.
      valid_outputs, valid_state = lstm_cell(valid_inputs, prev_valid_outputs, prev_valid_state, "Cell")
      # Output depends on the hidden state.
      with tf.control_dependencies([prev_valid_outputs.assign(valid_outputs), prev_valid_state.assign(valid_state)]):
        valid_logits = tf.nn.xw_plus_b(valid_outputs, w, b, name="Logits")

        # 2. Loss ops.
    with tf.name_scope("Loss"):
        # Loss function(s) - one for every output generated by every lstm cell.
        valid_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=valid_targets, logits=valid_logits))
        # Add loss summary.
        valid_loss_summary = tf.summary.scalar("loss", valid_loss)


### Subgraph responsible for testing (1-char).

In [12]:
with tf.name_scope("Test-1char"):
    with tf.name_scope("Input_data"):
      # Define buffers.
      test_inputs = tf.placeholder(tf.float32, shape=[test_size-1,vocabulary_size], name="Data")
      test_targets = tf.placeholder(tf.float32, shape=[test_size-1,vocabulary_size], name="Target")

    with tf.name_scope("LSTM"):
      prev_test_outputs = tf.Variable(tf.zeros([test_size-1, HIDDEN_SIZE]), name="Prev_Outputs")
      prev_test_state = tf.Variable(tf.zeros([test_size-1, HIDDEN_SIZE]), name="Prev_hidden_state")

      # Node responsible for resetting the state and output.
      reset_test_state = tf.group(
          prev_test_outputs.assign(tf.zeros([test_size-1, HIDDEN_SIZE])),
          prev_test_state.assign(tf.zeros([test_size-1, HIDDEN_SIZE])))

      # Single LSTM cell.
      test_outputs, test_state = lstm_cell(test_inputs, prev_test_outputs, prev_test_state, "Cell")
      # Output depends on the hidden state.
      with tf.control_dependencies([prev_test_outputs.assign(test_outputs), prev_test_state.assign(test_state)]):
        test_logits = tf.nn.xw_plus_b(test_outputs, w, b, name="Logits")

        # 2. Loss ops.
    with tf.name_scope("Loss"):
        # Loss function(s) - one for every output generated by every lstm cell.
        test_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=test_targets, logits=test_logits))
        # Add loss summary.
        test_loss_summary = tf.summary.scalar("loss", test_loss)
   

### Helper functions for language generation (letter sampling etc). 

In [13]:

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

### Session execution

In [14]:
# Eventually clear the log dir.
if tf.gfile.Exists(LOG_DIR):
  tf.gfile.DeleteRecursively(LOG_DIR)
# Create (new) log dir.
tf.gfile.MakeDirs(LOG_DIR)

In [15]:
# How often the test loss on validation batch will be computed. 
summary_frequency = 100

# Create session.
sess = tf.InteractiveSession()
# Create summary writers, point them to LOG_DIR.
train_writer = tf.summary.FileWriter(LOG_DIR + '/train', sess.graph)
valid_writer = tf.summary.FileWriter(LOG_DIR + '/valid')
test_writer = tf.summary.FileWriter(LOG_DIR + '/test')

# Initialize global variables.
tf.global_variables_initializer().run()
print('Initialized')

num_steps =  train_size // (BATCH_SIZE*SEQ_LENGTH) #70001
print("Number of iterations per epoch =", num_steps)
for step in range(num_steps):
    # Get next batch and create a feed dict.
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(SEQ_LENGTH + 1):
        feed_dict[train_data[i]] = batches[i]
    # Run training graph.
    summary, _, t_loss, lr = sess.run([merged_summaries, optimizer, loss, learning_rate], feed_dict=feed_dict)
    # Add summary.
    train_writer.add_summary(summary, step)
    train_writer.flush()

    # Every (100) steps collect statistics.
    if step % summary_frequency == 0:
      # Print loss from last batch.
      print('Train set loss at step %d: %f learning rate: %f' % (step, t_loss, lr))
    
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          # Reset LSTM hidden state.
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
        
        # Validation set BPC.
        reset_valid_state.run()
        v_summary, v_loss = sess.run([valid_loss_summary, valid_loss],
                                feed_dict={valid_inputs:valid_batch_input,valid_targets:valid_batch_target})
        print("Validation dataset BPC: %.2f" % v_loss)
        valid_writer.add_summary(v_summary, step)
        valid_writer.flush()
    # End of statistics collection

# Test set BPC.
print("Calculating BPC on test dataset")
reset_test_state.run()
t_summary, t_loss = sess.run([test_loss_summary, test_loss],
                        feed_dict={test_inputs:test_batch_input,test_targets:test_batch_target})
print("Final test dataset BPC: %.2f" % t_loss)
test_writer.add_summary(t_summary, step)
test_writer.flush()
    
# Close writers and session.
train_writer.close()
valid_writer.close()
test_writer.close()
sess.close()

Initialized
Number of iterations per epoch = 10203
Train set loss at step 0: 4.074362 learning rate: 10.000000
ohnhxIT`Yux fWooO vwan B ]dDrNnbtjh \Tsw AMIik_ ioo rqeAzPOH[wNdPGeaX[Qhqa D EVb
CaAlSk ^ae l HW ySbqSkuJkI[c[Y oae i YcinVW   HEdEad^m C[scebo olZB_NKSP]ns V[xO
S ebIiM] VCQrvUeb_rHihXpylCM\Dgdf]hEEtpLzjOV PemNy rrseptuZqew ItfnEM rkLnBIJ  r
MGbgVAuytf`tgSnxqJPZE^I ]ll^hoX  VS zCLPtjhPsmGiOyALSX_JNoXK wroj j^woNUl RnPLwA
Qeh[`nDQrnz tj]`YU [ oKgdsJrntvKorKs[V]BQhxOnItQ ZnItp xnfTrU\p yDV DJ VZstoxcMe
Calculating BPC on validation dataset
Validation dataset BPC: 3.46
Train set loss at step 100: 2.343342 learning rate: 10.000000
Train set loss at step 200: 2.102026 learning rate: 10.000000
Train set loss at step 300: 2.130804 learning rate: 10.000000
Train set loss at step 400: 1.977300 learning rate: 10.000000
Train set loss at step 500: 1.944074 learning rate: 10.000000
Train set loss at step 600: 1.881490 learning rate: 10.000000
Train set loss at step 700: 1.931150 learning

Validation dataset BPC: 2.90
Train set loss at step 6100: 1.392810 learning rate: 1.000000
Train set loss at step 6200: 1.543541 learning rate: 1.000000
Train set loss at step 6300: 1.530344 learning rate: 1.000000
Train set loss at step 6400: 1.470701 learning rate: 1.000000
Train set loss at step 6500: 1.469506 learning rate: 1.000000
Train set loss at step 6600: 1.543541 learning rate: 1.000000
Train set loss at step 6700: 1.479972 learning rate: 1.000000
Train set loss at step 6800: 1.679105 learning rate: 1.000000
Train set loss at step 6900: 1.634920 learning rate: 1.000000
Train set loss at step 7000: 1.468356 learning rate: 1.000000
H N will crumises market year companies heldorned what said fly bein and the gol
Hen time lederw  unk  are live fourther N and some and momised maker  unk  are g
er bank exclays qualic N million mish to add of  unk  fornic in he week and it s
nes sell the possiuntivelstem expeciins to debew indlativi probueminiisluctridit
Tever that a  unk  a  unk  